## Exploration of map reduce result

---

### Import Libraries

In [1]:
# import libraries
import findspark

# Locate the spark installation
findspark.init()

In [2]:
import pandas as pd
import pyspark as ps
from pyspark.sql.functions import col, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession


### Initialize Spark

In [7]:
# Initialize a SparkContext
spark = SparkSession.builder.appName("data_cleaning").getOrCreate()
spark.stop()
sc = ps.SparkContext(appName="prior_analysis")
# Initialize the Session
spark_session = ps.sql.SparkSession(sc)

23/09/08 11:37:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Connect and import data from HDFS directly into a Spark DataFrame

In [8]:
df_join_result = spark_session.read.csv('hdfs://localhost:9900/user/book_reviews/joined_tables_v2.csv', inferSchema=True, sep='\t')
df_join_result.printSchema()
df_join_result.describe().show()
df_join_result.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: timestamp (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)

+-------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+----+------------+------+------------------+--------------------+--------------------+--------------------+
|summary|                 _c0|                 _c1|               _c2|                 _c3|                 _c5|               _c6| _c7|         _c8|   _c9|              _c10|                _c11|                _c12|                _c13|
+-------+---------

In [10]:
tmp = df_join_result.limit(300).toPandas()
tmp.head(300)

# Title, description, authors, publisher, publishedDate, categories, Price, User_id, profileName, review/helpfulness, review/score, review/time, review/summary, review/text


/opt/homebrew/Cellar/apache-spark/3.4.1/libexec/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13
0,"@Large, Vol. 1","An Internet cafe and recording studio, known a...",['Ahmed Hoke'],TOKYOPOP,2019-12-20,['Comics & Graphic Novels'],9.99,None,"Del Keyes ""Elaborate Chattering Nut""",3/5,3.0,1072828800,"Talk is cheap, punk, and so is reading @large!",Pro illustrator Ahmed Hoke has really got my a...
1,"@Large, Vol. 1","An Internet cafe and recording studio, known a...",['Ahmed Hoke'],TOKYOPOP,2019-12-20,['Comics & Graphic Novels'],9.99,None,None,0/1,5.0,1073001600,Hip Hop and Art,@Large is an intellectual and artistic explora...
2,All Wired Up: Wire Techniques For the Beadwork...,The step-by-step techniques featured in this r...,['Mark Lareau'],Interweave,2000-10-01,['Crafts & Hobbies'],14.93,None,D. Riley,0/0,4.0,1185494400,All Wired Up,I like this book. The author put a lot of work...
3,All Wired Up: Wire Techniques For the Beadwork...,The step-by-step techniques featured in this r...,['Mark Lareau'],Interweave,2000-10-01,['Crafts & Hobbies'],14.93,None,SLG,14/15,5.0,1127952000,This book is great!!,I absolutely love this book! It has everything...
4,All Wired Up: Wire Techniques For the Beadwork...,The step-by-step techniques featured in this r...,['Mark Lareau'],Interweave,2000-10-01,['Crafts & Hobbies'],14.93,None,None,11/11,5.0,1042761600,Excellent book,"What a great instructive book -- clear, concis..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,23 Minutes In Hell: One Man's Story About What...,New York Times Best Seller and Over 1 million ...,['Bill Wiese'],Charisma Media,2010-09-24,['Religion'],10.39,None,birddog,1/2,5.0,1227571200,Good witnessing tool,This is one of the best books to give to your ...
296,23 Minutes In Hell: One Man's Story About What...,New York Times Best Seller and Over 1 million ...,['Bill Wiese'],Charisma Media,2010-09-24,['Religion'],10.39,None,Becky S. Curtis,1/2,5.0,1227571200,An Easy Way to Learn and Be Aware of Where You...,"""23 Minutes in Hell"" is remarkably easy to rea..."
297,23 Minutes In Hell: One Man's Story About What...,New York Times Best Seller and Over 1 million ...,['Bill Wiese'],Charisma Media,2010-09-24,['Religion'],10.39,None,T. Wall,1/2,5.0,1233532800,Yes Hell is Real,This book which expresses the experience of on...
298,23 Minutes In Hell: One Man's Story About What...,New York Times Best Seller and Over 1 million ...,['Bill Wiese'],Charisma Media,2010-09-24,['Religion'],10.39,None,T. Phelps,1/2,5.0,1245974400,Awesome,I received this book in a timely manner. I had...


In [ ]:
spark_session.stop()